In [8]:
# import subprocess
import os


This notebook prepares the data for the rural beauty model, by reprojecting, cropping and or resampling of raw datasets.

To do so, it loads paths from the rural_beauty/rural_beauty/config.py. 

It then either calls python functions from the same module or calls bash and R scripts from the scripts folder. 
To do the latter, variable are passed to the OS. 

In [9]:
%load_ext rpy2.ipython

# Beauty

Transform the beauty shapefiles to raster data.


In [11]:
# import paths from config.py
from rural_beauty.config import bild_vector_dir, bild_raster_dir # load absolute paths as defined in config.py

# pass them to the OS
os.environ['bild_vector_dir'] = str(bild_vector_dir)
os.environ['bild_raster_dir'] = str(bild_raster_dir)


In [14]:
!Rscript ../scripts/rasterize_ratings.R -i $bild_vector_dir -o $bild_raster_dir

Linking to GEOS 3.11.1, GDAL 3.6.4, PROJ 9.1.1; sf_use_s2() is TRUE
terra 1.7.78


# DEM

Take the digital elevation model and calculate the range of elevation in the 1x1 km grid. 

In [28]:
# import paths from config.py
from rural_beauty.config import DEM_EU, DEM_DE_range

# pass them to the OS
os.environ['DEM_EU'] = str(DEM_EU)
os.environ['DEM_DE_range'] = str(DEM_DE_range)

In [22]:
!../scripts/process_dem_data_minmax.sh $DEM_EU $DEM_DE_range

Creating output file that is 642P x 867L.
Processing /media/sf_Granular/beauty/data/raw/dem/EU_DEM_EU.tif [1/1] : 0Using internal nodata values (e.g. -9999) for image /media/sf_Granular/beauty/data/raw/dem/EU_DEM_EU.tif.
Copying nodata values from source /media/sf_Granular/beauty/data/raw/dem/EU_DEM_EU.tif to destination EU_DEM_DE_max.tif.
...10...20...30...40...50...60^C


In [ ]:
# TODO clean up __deletable_ min and max in case the previous step failed.

# Process the OSM data. 

We downloaded the whole osm data from geofabrik on July 3rd from:

https://download.geofabrik.de/europe-latest.osm.pbf

https://download.geofabrik.de/germany-latest.osm.pbf


In [6]:
# import paths from config.py
from rural_beauty.config import OSM_full_DE
from rural_beauty.config import powerlines_DE_vector, powerlines_DE_raster
from rural_beauty.config import streets_DE_vector, streets_DE_raster
from rural_beauty.config import windpower_DE_vector, windpower_DE_raster

import os
# pass them to the OS
os.environ['powerlines_DE_vector'] = str(powerlines_DE_vector)
os.environ['powerlines_DE_raster'] = str(powerlines_DE_raster)
os.environ['streets_DE_vector'] = str(streets_DE_vector)
os.environ['streets_DE_raster'] = str(streets_DE_raster)
os.environ['windpower_DE_vector'] = str(windpower_DE_vector)
os.environ['windpower_DE_raster'] = str(windpower_DE_raster)

Subset the big .pbf file to windpower, streets, and powerlines

In [ ]:
!../scripts/osm_extract_streets.sh $OSM_full_DE $streets_DE_vector
!../scripts/osm_extract_powerlines.sh $OSM_full_DE $powerlines_DE_vector
!../scripts/osm_extract_windpower.sh $OSM_full_DE $windpower_DE_vector

In [2]:
!../scripts/rasterize_OSM_line_geom.R $powerlines_DE_vector $powerlines_DE_raster
!../scripts/rasterize_OSM_line_geom.R $streets_DE_vector $streets_DE_vector
!../scripts/rasterize_OSM_point_geom.sh $windpower_DE_vector $windpower_DE_raster

terra 1.7.78
Rasterization complete. Output saved to: /media/sf_Granular/beauty/data/processed/osm/len_powerlines_DE_4647.tif 
terra 1.7.78
Error: [writeRaster] cannot guess file type from filename
Execution halted
0...10...20...30...40...50...60...70...80...90...100 - done.
Rasterization complete. Output saved to: /media/sf_Granular/beauty/data/processed/osm/freq_windpowerplants_DE_4647.tif


# Corine Landcover (CLC)

In [1]:
# import paths from config.py
from rural_beauty.config import CLC_DE, CLC_boolean_layers_DE_dir, CLC_coverage_DE_dir
from rural_beauty import split_CLC_layers

split_CLC_layers.main(CLC_DE, CLC_boolean_layers_DE_dir)

os.listdir(CLC_boolean_layers_DE_dir)


This is the rural_beauty module


['code_acker.tif',
 'code_heide.tif',
 'code_natgru.tif',
 'code_noveg.tif',
 'code_obst.tif',
 'code_seemee.tif',
 'code_spfr.tif',
 'code_stoer.tif',
 'code_wald.tif']

In [2]:
# pass the coverage output folder and the boolean layers folder to the OS for the next step
os.environ['CLC_boolean_layers_DE_dir'] = str(CLC_boolean_layers_DE_dir)
os.environ['CLC_coverage_DE_dir'] = str(CLC_coverage_DE_dir)


In [4]:
!../scripts/resample_CLC_DE.sh $CLC_boolean_layers_DE_dir $CLC_coverage_DE_dir

File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_acker.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_heide.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_natgru.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_noveg.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_obst.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_seemee.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_spfr.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_stoer.tif already exists, skipping...
File /media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/code_wald.tif already exists, skipping...
Resampling complete.

# Hemerobieindex


In [2]:
from rural_beauty.config import heme2012_DE, heme2012_DE_repojected

os.environ['heme2012_DE'] = str(heme2012_DE)
os.environ['heme2012_DE_repojected'] = str(heme2012_DE_repojected)

In [6]:
%%bash 
echo "Input: $heme2012_DE"
echo "Output: $heme2012_DE_repojected"

../scripts/reproject_heme.sh $heme2012_DE $heme2012_DE_reprojected

Input: /media/sf_Granular/beauty/data/raw/hemerobie_IOER/heme2012.tif
Output: /media/sf_Granular/beauty/data/cleaned/hemerobie_IOER/heme2012.tif


Creating output file that is 642P x 867L.
Processing /media/sf_Granular/beauty/data/raw/hemerobie_IOER/heme2012.tif [1/1] : 0Using internal nodata values (e.g. -9998) for image /media/sf_Granular/beauty/data/raw/hemerobie_IOER/heme2012.tif.
Copying nodata values from source /media/sf_Granular/beauty/data/raw/hemerobie_IOER/heme2012.tif to destination /media/sf_Granular/beauty/data/cleaned/hemerobie_IOER/heme2012.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
